In [11]:
import pandas as pd
import os
import torch 
import clip
import numpy as np
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device,"device")

cuda device


In [12]:
summary_bert_path = "/users/eleves-b/2022/hanna.mergui/Computer-Vision/ComputerVision_Data/Summaries/export_summary_Bert.csv"
summary_bert = pd.read_csv(summary_bert_path)

data_dir = "ComputerVision_Data"
images_dir = "Food Images/Food Images"

In [13]:
recipe = summary_bert["Summary"]
print("number of sumary recipe ",len(recipe))

images = os.listdir(os.path.join(data_dir, images_dir))
print("number of images ",len(images))

number of sumary recipe  13463
number of images  3486


In [14]:
model_name = 'RN50'
model, preprocess = clip.load(model_name, device)
input_resolution = model.visual.input_resolution
context_length = model.context_length
vocab_size = model.vocab_size

TypeError: 'torch._C.Node' object is not subscriptable

In [ ]:
""" 
#Tensor creation 

# Creation of a tensor for the text 
device = "cuda" if torch.cuda.is_available() else "cpu"
#print(device,"device")
text_tensor = torch.zeros(len(summary_bert), context_length, dtype=torch.long)
labels = []
for i, row in enumerate(summary_bert.iterrows()):
    summary = row[1]["Summary"]
    if i==0: 
        print(summary)
        summary = summary[:context_length]  # Truncate the text if it's too long
    text_tensor[i] = clip.tokenize([summary],truncate=True).to(device)  # Access row data using integer indices
    labels.append(row[1]["Image_Name"])  # Access row data using integer indices

"""

#torch.save(text_tensor, "text_tensor.pt")

Roast chicken in a large cast-iron skillet until an instant-read thermometer inserted into the thickest part of breast registers 155°F, 50–60 minutes. Meanwhile, roast squash on lower rack until mostly tender, about 25 minutes.


In [9]:
text_tensor = torch.load("text_tensor.pt")

In [10]:
images_inputs=[]

for im in images:
    im_path = os.path.join(data_dir, images_dir, im)
    images_inputs.append(preprocess(Image.open(im_path)))
images_inputs_tensor = torch.tensor(np.stack(images_inputs)).to(device)

NameError: name 'preprocess' is not defined

In [ ]:
with torch.no_grad():
    image_features = model.encode_image(images_inputs_tensor.to(device))
    text_features = model.encode_text(text_tensor.to(device))


RuntimeError: CUDA error: no kernel image is available for execution on the device

In [ ]:
image = preprocess(Image.open("ComputerVision_Data/Food Images/Food Images/-bloody-mary-tomato-toast-with-celery-and-horseradish-56389813.jpg")).unsqueeze(0).to(device)
text = clip.tokenize(["a diagram", "a dog", "a cat"]).to(device)

with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    
    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

print("Label probs:", probs)  # prints: [[0.9927937  0.00421068 0.00299572]]

RuntimeError: CUDA error: no kernel image is available for execution on the device